# 뉴럴 네트워크 학습 알고리즘 구현

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
import tensorflow as tf

## 하이퍼파라미터 정의

In [3]:
EPOCHS = 10

## 네트워크 구조 정의

In [4]:
class MyModel(tf.keras.Model):
  def __init__(self):
      super().__init__()
      self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
      self.dense1 = tf.keras.layers.Dense(32, activation='relu')
      self.dense2 = tf.keras.layers.Dense(64, activation='relu')
      self.dense3 = tf.keras.layers.Dense(128, activation='relu')
      self.dense4 = tf.keras.layers.Dense(256, activation='relu')
      self.dense5 = tf.keras.layers.Dense(10, activation='softmax')
      
  def call(self, x, training=None, mask=None):
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    return self.dense5(x)

## 학습 함수 구현

In [6]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
  with tf.GradientTape() as tape: # gradient 계산
    predictions = model(images) # batchsize * 10(Classes)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

# 테스트 함수 구현

In [7]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
  predictions = model(images)
  loss = loss_object(labels, predictions)

  test_loss(loss)
  test_accuracy(labels, predictions)

## 데이터 불러오기

In [8]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.04

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

11501568/11490434 [==============================] - 0s 0us/step


## 모델 생성

In [9]:
model = MyModel()

## 손실함수 및 최적화 알고리즘 정의

In [11]:
print(y_train)
print(y_train.shape)

[5 0 4 ... 5 6 8]
(60000,)


In [12]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 성능 지표 정의

In [13]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 구현

In [16]:
for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for images, labels in test_ds:
    test_step(model, images, labels, loss_object, test_loss, test_accuracy)

  template = f'Epoch {epoch+1}, Loss: {train_loss.result()}, Accuracy: {train_accuracy.result()*100}, Test loss: {test_loss.result()}, Test Accuracy: {test_accuracy.result()*100}'
  print(template)

  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

Epoch 1, Loss: 0.21669098734855652, Accuracy: 93.42583465576172, Test loss: 0.16863949596881866, Test Accuracy: 95.23500061035156
Epoch 2, Loss: 0.11479869484901428, Accuracy: 96.51166534423828, Test loss: 0.16404791176319122, Test Accuracy: 95.4800033569336
Epoch 3, Loss: 0.09563740342855453, Accuracy: 97.0, Test loss: 0.1352071315050125, Test Accuracy: 96.10000610351562
Epoch 4, Loss: 0.08114465326070786, Accuracy: 97.40999603271484, Test loss: 0.14508280158042908, Test Accuracy: 95.95000457763672
Epoch 5, Loss: 0.06860923022031784, Accuracy: 97.79999542236328, Test loss: 0.1463647186756134, Test Accuracy: 96.19000244140625
Epoch 6, Loss: 0.06190233677625656, Accuracy: 98.01667022705078, Test loss: 0.12597252428531647, Test Accuracy: 96.77000427246094
Epoch 7, Loss: 0.05496089905500412, Accuracy: 98.26333618164062, Test loss: 0.13177867233753204, Test Accuracy: 96.86000061035156
Epoch 8, Loss: 0.05491945520043373, Accuracy: 98.22833251953125, Test loss: 0.1504707634449005, Test Accur